<a href="https://colab.research.google.com/github/mohammadalmalt/AI/blob/main/MusicGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U git+https://github.com/facebookresearch/audiocraft.git
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install scipy numpy


In [5]:
import torch
from audiocraft.models import MusicGen
from audiocraft.utils.notebook import display_audio
import scipy.io.wavfile
import numpy as np

# Check environment and load model
device = "cuda" if torch.cuda.is_available() else "cpu"

model = MusicGen.get_pretrained("facebook/musicgen-large")
print("MusicGen model loaded!")

# Define prompt and generation parameters
prompt = ["sad romantic music played on harp"]
model.set_generation_params(duration=15)  # 15 seconds of music

# Generate music (output as float32 in [-1, 1])
print("Generating music...")
generated_audio = model.generate(prompt, progress=True)
music_waveform = generated_audio[0].cpu().numpy()  # still in float32

# Listen in Colab (using the float32 waveform)
display_audio(torch.tensor(music_waveform), sample_rate=model.sample_rate)

# --- Saving the audio correctly ---
output_path = "generated_music2.wav"

# Ensure the data is a NumPy array and remove any extra dimensions
music_waveform = np.squeeze(music_waveform)

# If stereo with shape [2, samples], transpose to [samples, 2]
if music_waveform.ndim == 2 and music_waveform.shape[0] == 2:
    music_waveform = music_waveform.T

# Clip to ensure values are within [-1, 1] (should already be, but just in case)
music_waveform = np.clip(music_waveform, -1, 1)

# Convert float32 [-1,1] to int16 [-32768, 32767] **once**
music_waveform_int16 = (music_waveform * 32767).astype(np.int16)

# Use the sample rate from the model (or default to 32000)
sample_rate = int(model.sample_rate) if hasattr(model, "sample_rate") else 32000

# Save the WAV file
scipy.io.wavfile.write(output_path, sample_rate, music_waveform_int16)
print(f"High-quality music saved as: {output_path}")


/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


MusicGen model loaded!
Generating music...


High-quality music saved as: generated_music2.wav


In [ ]:
import torch
from audiocraft.models import MusicGen
from audiocraft.utils.notebook import display_audio
import scipy.io.wavfile
import numpy as np

# Check environment and load model
device = "cuda" if torch.cuda.is_available() else "cpu"

# Use a larger model for better quality
model = MusicGen.get_pretrained("facebook/musicgen-large")
print("MusicGen model loaded!")

# Define prompt and generation parameters
prompt = ["calm and relaxing chillout music with soft piano and ambient pads, 80 bpm"]
model.set_generation_params(
    duration=30,  # 30 seconds of music
    temperature=1.5,  # Slightly more creative
    top_k=250,  # Limit to top 250 tokens
    top_p=0.95,  # Use nucleus sampling
)

# Generate music (output as float32 in [-1, 1])
print("Generating music...")
generated_audio = model.generate(prompt, progress=True)
music_waveform = generated_audio[0].cpu().numpy()  # still in float32

# Listen in Colab (using the float32 waveform)
display_audio(torch.tensor(music_waveform), sample_rate=model.sample_rate)

# --- Saving the audio correctly ---
output_path = "chill_music.wav"

# Ensure the data is a NumPy array and remove any extra dimensions
music_waveform = np.squeeze(music_waveform)

# If stereo with shape [2, samples], transpose to [samples, 2]
if music_waveform.ndim == 2 and music_waveform.shape[0] == 2:
    music_waveform = music_waveform.T

# Clip to ensure values are within [-1, 1] (should already be, but just in case)
music_waveform = np.clip(music_waveform, -1, 1)

# Convert float32 [-1,1] to int16 [-32768, 32767] *once*
music_waveform_int16 = (music_waveform * 32767).astype(np.int16)

# Use the sample rate from the model (or default to 32000)
sample_rate = int(model.sample_rate) if hasattr(model, "sample_rate") else 32000

# Save the WAV file
scipy.io.wavfile.write(output_path, sample_rate, music_waveform_int16)
print(f"High-quality music saved as: {output_path}")
